# RGI05 (Greenland Periphery)

F. Maussion, S. Galos & F. Roura-Adseiras

Goal: same as RGI6 except in Northeast with an update from Zürich.

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
from utils import mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist
import os

## Files and storage paths

In [ ]:
# Region of interest
reg = 5

# go down from rgi7_scripts/workflow
data_dir = '../../rgi7_data/'

# Level 2 GLIMS files
l2_dir = os.path.join(data_dir, 'l2_sel_reg_tars')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_rgi7a'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_rgi7a_tar'))

# RGI v6 file for comparison later 
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', '05_rgi60_GreenlandPeriphery.zip')

### Load the input data

In [ ]:
# Read L2 files
shp = gpd.read_file('tar://' + l2_dir + f'/RGI{reg:02d}.tar.gz/RGI{reg:02d}/RGI{reg:02d}.shp')

### List of submissions 

In [ ]:
sdf, _ = submission_summary(shp)
sdf

Notes based on individual submission evaluations: 

- 580: This is the RGI
- 729: updated from a region in NE

In [ ]:
# # Write out selection in intermediate shape files for manual GIS review
# tmp_output_dir = mkdir(os.path.join(data_dir, 'l0_tmp_data', f'rgi{reg:02d}_inventories'))
# tmp_output_dir_tar = mkdir(os.path.join(data_dir, 'l0_tmp_data'))
# for subid in shp.subm_id.unique():
#     s_loc = shp.loc[shp.subm_id == subid]
#     s_loc.to_file(tmp_output_dir + f'/subm_{int(subid):03d}.shp')
# print('Taring...')
# print(subprocess.run(['tar', '-zcvf', f'{tmp_output_dir_tar}/rgi{reg:02d}_inventories.tar.gz', '-C', 
#                       os.path.join(data_dir, 'l0_tmp_data'), f'rgi{reg:02d}_inventories']))

### Apply selection criteria to create the RGI7 data subset

In [ ]:
# We use 'subm_id' = 580 as our main dataset
RGI_ss_580 = shp.loc[shp['subm_id'] == 580].copy()
RGI_ss_580['is_rgi6'] = True

# The subm_id = 729 will be added to subm_id=580 after some actions
RGI_ss_729 = shp.loc[shp['subm_id'] == 729].copy()
RGI_ss_729['is_rgi6'] = False

In [ ]:
# Read box containing glaciers with subm_id = 729
from utils import open_zip_shapefile
rgi_729_box = data_dir + f'/l0_support_data/rgi{reg:02d}_north_outline_sel_box.zip'
box = open_zip_shapefile(rgi_729_box)

In [ ]:
# Plot correct glaciers and shapefile
base = box.plot(color='none', edgecolor='black')
RGI_ss_729.geometry.plot(ax=base, color='C0', markersize=5);

We see that for sumb_id = 729 all glaciers fall inside the box polygon.

Now we remove the glaciers inside and add the RGI_ss_729 to RGI_ss:

In [ ]:
# Save glacier centroid in rp:
rp = RGI_ss_580.representative_point()

rp = rp.to_frame('geometry')
rp['orig_index'] = RGI_ss_580.index

# Find non overlying polygons
difference = gpd.overlay(rp, box, how='difference')

# Create temporary variable
RGI_ss_580 = RGI_ss_580.loc[difference['orig_index']]

In [ ]:
# Merge original RGI 580 (without the glacier in the box) and the RGI 729
rgi7 = pd.concat([RGI_ss_580, RGI_ss_729])

### Some sanity checks 

In [ ]:
needs_size_filter(rgi7)

In [ ]:
sdf, df_class = submission_summary(rgi7)
df_class

In [ ]:
# Check the orphaned rock outcrops
orphan_f = os.path.join(data_dir, 'l1_orphan_interiors', f'RGI{reg:02d}', f'RGI{reg:02d}.shp')
if os.path.exists(orphan_f):
    orphan_f = gpd.read_file(orphan_f)
    if np.any(np.isin(rgi7.subm_id.unique(), orphan_f.subm_id.unique())):
        print('Orphan rock outcrops detected.')

### Plots 

In [ ]:
plot_map(rgi7, reg, aspect=2, loc='lower right')

In [ ]:
plot_map(rgi7, reg, is_rgi6=True, aspect=2, loc='lower right')

In [ ]:
plot_date_hist(rgi7, reg)

### Text for github

In [ ]:
fgh = sdf.T
fgh

In [ ]:
print(fgh.to_markdown(headers=np.append(['subm_id'], fgh.columns)))

## Write out and tar 

In [ ]:
dd = mkdir(f'{output_dir}/RGI{reg:02d}/', reset=True)

print('Writing...')
rgi7.to_file(dd + f'RGI{reg:02d}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI{reg:02d}.tar.gz', '-C', output_dir, f'RGI{reg:02d}']))

## New RGI-file created - Check result!

### load reference data (here RGI6 and Philipp) to enable comparison

In [ ]:
# load reference data
ref_rgi6 = open_zip_shapefile(rgi6_reg_file)
ref_pr = open_zip_shapefile(data_dir + '/l0_support_data/c3s_gi_rgi05_aero_1978.zip')

### Number of elements (differences do not depict problems)

In [ ]:
print('Number of glaciers in new RGI:', len(rgi7))
print('Number of glaciers in RGI6:', len(ref_rgi6))
print('Difference:', len(rgi7)-len(ref_rgi6))

### How many nominal glaciers were there in RGI06?

In [ ]:
len(ref_rgi6.loc[ref_rgi6.Status == 2])

### Total area

In [ ]:
# add an area field to RGI_ss and reference data
ref_rgi6['area'] = ref_rgi6.to_crs({'proj':'cea'}).area
ref_pr['area'] = ref_pr.to_crs({'proj':'cea'}).area

### Northern Greenland (729, Rastner, no problem there):  

In [ ]:
print('Number of glaciers in RGI7 subset:', len(RGI_ss_729))
print('Number of glaciers in reference data (FP):', len(ref_pr))
print('Difference:', len(RGI_ss_729)-len(ref_pr))

In [ ]:
# print and compare area values
Area_7 = RGI_ss_729['area'].sum() * 1e-6
print('Area RGI7 [km²]:', Area_7)
Area_6 = ref_pr['area'].sum() * 1e-6
print('Area RGI6 [km²]:', Area_6)
d = (Area_7 - Area_6)
print('Area difference [km²]:', d)

This is brilliant! No issue there.

### Rest of Greenland (problems):  

In [ ]:
# Save glacier centroid in rp:
rp = ref_rgi6.representative_point()

rp = rp.to_frame('geometry')
rp['orig_index'] = ref_rgi6.index

# Find non overlying polygons
difference = gpd.overlay(rp, box, how='difference')

# Create temporal variable
ref_rgi6_sel = ref_rgi6.loc[difference['orig_index']]

In [ ]:
needs_size_filter(ref_rgi6_sel)

In [ ]:
print('Number of glaciers in RGI7 subset:', len(RGI_ss_580))
print('Number of glaciers in reference data (FP):', len(ref_rgi6_sel))
print('Difference:', len(RGI_ss_580)-len(ref_rgi6_sel))

In [ ]:
# print and compare area values
Area_7 = RGI_ss_580['area'].sum() * 1e-6
print('Area RGI7 [km²]:', Area_7)
Area_6 = ref_rgi6_sel['area'].sum() * 1e-6
print('Area RGI6 [km²]:', Area_6)
d = (Area_7 - Area_6)
print('Area difference [km²]:', d)

Large issue with rock outcrops. Reported here: https://github.com/GLIMS-RGI/rgi7_scripts/issues/11